In [39]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline



# gloabale var: image

In [40]:


def load_image(file_name, width=1280, height=720):
    # load the image
    image = cv2.imread('images/' + file_name)
#     image = cv2.imread(file_name)
    
    if image is not None:
        # resize to specific resolution
        image = cv2.resize(image, (width, height))
        # change it to RGB image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        return image
    else:
        print("image not loaded")
        
#convert pic to binary          
def rgb_select(image, thresh=(0, 255), color='r'):
    if color == 'b':
        color_channel = image[:,:,2]
    elif color == 'r':
        color_channel = image[:,:,0]
    binary_output = np.zeros_like(color_channel)
    binary_output[(color_channel > thresh[0]) & (color_channel <= thresh[1])] = 1
    return binary_output





In [41]:
# filter 1
# input list to store required ellipse, all contours from image
# only choose contours' area that are greater than 40
# fitEllipse 
# will return a rotated rect
# ellipse--> ((xc,yc),(a,b),theta)
        #xc : x coordinate of the center
        # yc : y coordinate of the center
        # a : width
        # b : height
        # theta : rotation angle
# output list that stored required ellipse
def get_ellipse(all_element_list,contours):
    
   
    
    
    for i in contours: 
    
        area = cv2.contourArea(i)
        if area > 50:
            ellipse = cv2.fitEllipse(i) 
            all_element_list.append(ellipse)
            
            ################
            cv2.ellipse(temp,ellipse,(255,0,222),2) # draw contours to pic "temp"
#     plt.figure(figsize=(100,80))    
#     plt.imshow(temp)
      
    
    # list of ellipses which area are greater than 
    return all_element_list
    
    

In [42]:
def slope(x1, y1, x2, y2):
    return (y2-y1)/(x2-x1)


In [43]:
# pair up ellipses
# input: list to store required pairs, list of ellipse that pass perivous fileter
# output: list of pairs of ellipses
# [(elp1,elps2),(elps3,elps4)....]
# ellipses have similar width that are similar, within 5
# ellipses have similar rotation angle: after substraction, within -5,5 or not in -175,175
# ellipses x distances are in range of (60,300) 
def get_pairs(pairs,all_element_list):
    
    ################
    
    itr = len(all_element_list)-1
    for i in range(0, itr):
#         print("angle: ",int(all_element_list[i][2]))
#         if int(all_element_list[i][2]) == xxxx:
#             cv2.ellipse(temp,all_element_list[i],(255,255,250),3)
    
        
        for j in range(i+1,itr+1):
            
             
            # angle within 5 degree   
#             print("angle differene before",int(all_element_list[i][2]-all_element_list[j][2]))
            if (int(all_element_list[i][2]-all_element_list[j][2]) in range(-10,10)) or (int(all_element_list[i][2]-all_element_list[j][2]) not in range(-170,170)): 


# #               x distance between centers inrange 60,300
                if (abs(int(all_element_list[i][0][0]-all_element_list[j][0][0])) in range(60,180)) and (abs(int(all_element_list[i][0][1]-all_element_list[j][0][1]) ) in range(0,80) ):

                    
#                     length two ellipse's semi axes are similar, ie the height is similar
                    if(abs(int(all_element_list[i][1][1]-all_element_list[j][1][1])) < 15) or ((abs(int(all_element_list[i][1][1]-all_element_list[j][1][1])) in range (14,24) ) and int(all_element_list[i][1][1]+all_element_list[j][1][1])<90 ) :
                        
            
                        # paired up ellipse's rotation angle should not be similar to their centers' connection slope
                        slp = slope(all_element_list[i][0][0],all_element_list[i][0][1],all_element_list[j][0][0],all_element_list[j][0][1])
#                         print("X center : ",all_element_list[i][0][0],all_element_list[i][0][1])
#                         print("Y center : ",all_element_list[j][0][0],all_element_list[j][0][1])
                        deg_referX = np.arctan(slp)*57.3 # degree of slope reference x axis
                        
                        deg = 90 + deg_referX 
                        
#                         print("degRx: ", deg_referX)
#                         print("deg: ", deg)
#                         print("i angle : ",all_element_list[i][-1])
#                         print("j angle : ",all_element_list[j][-1])
                        if((abs(deg - all_element_list[i][-1] ) > 30 ) and (abs(deg - all_element_list[j][-1] ) > 30 )  ):
#                         if int(deg_referX) != -40:
                            

                            pairs.append([all_element_list[i],all_element_list[j]])
                            


                            cv2.ellipse(temp,all_element_list[i],(0,200,200),3)
                            cv2.ellipse(temp,all_element_list[j],(0,200,200),3)
#     plt.figure(figsize=(100,80))    
#     plt.imshow(temp)
    print("pairs len: ",len(pairs))
    
    return pairs
    
    

In [44]:
# limit to only one pair
# get the mid point coordinate between two ellipse centers in a pair
# return the mid points' coor in aim_list
 
def get_aim(aim_list,pairs):
    
    # equal to sum of heights of ellipse in a pair
    maxSum = 0
    x =0 
    y =0
    for i in pairs:
        if maxSum < (i[0][1][1]+i[1][1][1]):
            maxSum = i[0][1][1]+i[1][1][1]
            x = int((i[0][0][0]+i[1][0][0])/2)
            y = int((i[0][0][1]+i[1][0][1])/2)
    

    aim =(x,y)
    aim_list.append(aim)
        
    return aim_list
    
    
    

In [45]:
def enhanced_get_aim(aim_list,all_element_list):
    ellipse_list = []
    for i in all_element_list:
        
        if int(i[-1]) not in range(20,165):
            cv2.ellipse(temp,i,(0,200,200),3)
            ellipse_list.append(i)
    
    if len(ellipse_list)>1:
        maxArea = 0
        for i in ellipse_list:
            if maxArea < i[1][1] * i [1][0]:
                maxArea = i[1][1] * i [1][0]
                if i[-1] < 90:
                    elp_center = (i[0][0]-40, i[0][1]-15)
                else:
                    elp_center = (i[0][0]+40, i[0][1]-15)
                    
                
    else:
        if ellipse_list[0][-1] < 90:
            elp_center = (ellipse_list[0][0][0]-40, ellipse_list[0][0][1]-15)
        else:
            elp_center = (ellipse_list[0][0][0]+40, ellipse_list[0][0][1]-15)
                    
            
    
    target = (int(elp_center[0]),int(elp_center[1]))
    aim_list.append(target)
    
    return aim_list

In [46]:
# main function for aiming target
# input: image from camera
# output: list of all targets coordinates: aim_list

def process_image(image):
    
    # var: 
    # contours: original contours select from binary pic
    # all_element_list: ellipses that passed filters(restrictions)


    #convert to binary pic with only red 
    rgb_binary = rgb_select(image, thresh=(66, 255), color='r')
    
    #im2: binary pic with contours drawn
    #contours: list of groups of points of contour
    im2, contours, hierarchy = cv2.findContours(rgb_binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    
    all_element_list = []
    
    all_element_list = get_ellipse(all_element_list,contours)
    
    # use list pairs to store all desired paired up ellipse
    pairs = []
    
    pairs = get_pairs(pairs,all_element_list)
#     print("pairs: ",pairs)
    
    
    # list that stores all target coordinates
    aim_list = []
    
    
    # @TODO
#   if aim_list == none
#   then aim_list = enhanced_get_aim(....), # like: try to use single light to get target, or conbine with target cycle
    
    
    
    if len(pairs) == 0:
        aim_list = enhanced_get_aim(aim_list,all_element_list)
        
    else:
        aim_list = get_aim(aim_list,pairs)
  
    return aim_list
        
    
    
        
    
    




In [47]:
# multi pic test area



print("start")
y = 1
for x in range(1,494):
    
    
    picName = 'sentry_view' + str(x) + '.png'
    image = load_image(picName)
    temp = image.copy()

    # ideal output: targets' coor
    all_target = process_image(image)

    # test and print target on temp pic
    for i in all_target:
        print(all_target)
        cv2.circle(temp,(i[0],i[1]), 20, (0,0,255), -1)
        cv2.putText(temp,str(x),(500,500), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (0, 255, 255))
        
    newFileName = str(x) + '.png'
    cv2.imwrite(newFileName,temp)
#     plt.figure(figsize=(30,10))    
#     plt.imshow(temp)
print("end")  

start
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(541, 446)]
pairs len:  1
[(531, 452)]
pairs len:  1
[(515, 458)]
pairs len:  1
[(514, 457)]
pairs len:  1
[(512, 457)]
pairs len:  1
[(517, 456)]
pairs len:  1
[(515, 456)]
pairs len:  1
[(516, 456)]
pairs len:  1
[(516, 456)]
pairs len:  1
[(516, 456)]
pairs len:  1
[(491, 464)]
pairs len:  1
[(492, 464)]
pairs len:  1
[(485, 468)]
pairs len:  1
[(485, 469)]
pairs len:  1
[(485, 469)]
pairs len:  1
[(485, 469)]
pairs len:  1
[(481, 470)]
pairs len:  1
[(475, 471)]
pairs len:  1
[(452, 476)]
pairs len:  1
[(447, 477)]
pairs len:  1
[(447, 477)]
pairs len:  1
[(447, 477)]
pairs len:  1
[(447, 477)]
pairs len:  1
[(447, 477)]
pairs len:  1
[(447, 477)]
pairs len:  1
[(444, 477)]
pairs len:  1
[(438, 478)]
pairs len:  1
[(404, 4

pairs len:  1
[(430, 487)]
pairs len:  1
[(429, 487)]
pairs len:  1
[(429, 487)]
pairs len:  1
[(429, 487)]
pairs len:  1
[(438, 486)]
pairs len:  1
[(447, 485)]
pairs len:  1
[(481, 478)]
pairs len:  1
[(483, 478)]
pairs len:  1
[(484, 478)]
pairs len:  1
[(483, 478)]
pairs len:  1
[(541, 455)]
pairs len:  1
[(540, 457)]
pairs len:  1
[(540, 456)]
pairs len:  1
[(537, 457)]
pairs len:  1
[(537, 457)]
pairs len:  1
[(308, 535)]
pairs len:  1
[(331, 540)]
pairs len:  1
[(333, 539)]
pairs len:  1
[(334, 539)]
pairs len:  1
[(332, 538)]
pairs len:  1
[(332, 538)]
pairs len:  1
[(332, 538)]
pairs len:  1
[(406, 539)]
pairs len:  1
[(408, 538)]
pairs len:  1
[(399, 538)]
pairs len:  1
[(398, 538)]
pairs len:  1
[(398, 538)]
pairs len:  1
[(398, 538)]
pairs len:  1
[(398, 538)]
pairs len:  1
[(398, 538)]
pairs len:  2
[(423, 535)]
pairs len:  1
[(501, 523)]
pairs len:  1
[(502, 523)]
pairs len:  1
[(504, 521)]
pairs len:  1
[(503, 522)]
pairs len:  1
[(504, 522)]
pairs len:  1
[(600, 479)]
p

pairs len:  0
i detail:   ((1064.662109375, 603.8237915039062), (9.255918502807617, 56.6582145690918), 157.31454467773438)
i detail:   ((622.5982055664062, 590.23291015625), (15.516939163208008, 35.77271270751953), 167.0802001953125)
i detail:   ((361.60894775390625, 574.875732421875), (15.144275665283203, 36.03703689575195), 11.800263404846191)
i detail:   ((598.1349487304688, 432.82275390625), (5.881777286529541, 18.877777099609375), 151.03042602539062)
i detail:   ((216.08724975585938, 401.1404724121094), (11.83983325958252, 17.1306095123291), 95.76964569091797)
i detail:   ((277.7476806640625, 394.1383056640625), (8.82834529876709, 42.86823272705078), 67.11307525634766)
i detail:   ((275.6714782714844, 375.4359130859375), (19.124547958374023, 80.74842071533203), 69.77018737792969)
[(662, 575)]
pairs len:  0
i detail:   ((1064.7044677734375, 604.1502685546875), (9.349230766296387, 55.45042419433594), 156.76998901367188)
i detail:   ((622.5486450195312, 590.1762084960938), (15.364222